#### PIPE - 9:    
This pipe covers the following : 
1. All features of pipe 8
2. CSV File integration

NOTE : We will be also defining the final strucut of json need for capturing violation in the following file

Order of entried in json

violation_type_id - ADDED    
vehicle_type_id - ADDED    
vehicle_speed - ADDED    
travel_direction - ADDED    
location_id - ADDED    
camera_id - ADDED    
evidence_image_url - ADDED    
numbe_plate_ocr_image_url  - ADDED     
evidence_video_url   - ADDED     
numbe_plate_image_url - ADDED    
scanned_number_plate_number - ADDED         
violation_date   - ADDED    
violation_time   - ADDED    

##### Strucutre of JSON
main_json={
"tracking_id":{ and the rest entries}
}

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
import threading 
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *


ssd=[
     {'id': 3, 'name': 'car', 'vehicle_class': 'lmv'},
     {'id': 4, 'name': 'motorcycle', 'vehicle_class': 'mcwg/mcg'},
     {'id': 5, 'name': 'truck', 'vehicle_class': 'hgmv'},
     {'id': 6, 'name': 'bus', 'vehicle_class': 'hpmv'},
     {'id': 7, 'name': 'rikshaw', 'vehicle_class': 'l5m'}
    ]
voi=[
   {'id': 2, 'name': 'over_speed'}, 
   {'id': 3, 'name': 'no_helmet'}
]
vehicle_class_json=violation_based_json_creator(ssd)
violation_type_json=violation_based_json_creator(voi)
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines


counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}

main_violation_tracker_json={}
x_start=300
x_end=800
y_start=198
y_end=280
#This inputs will be directly taken from front_end
location_id=-1
camera_id=-1
frame_count=0
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'

In [2]:
start_time=time.time()
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('models/yolov8l.pt')
yolo_model_classes=model.names

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')
current_date=get_current_date()

while True:    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    resized_image=cv2.resize(frame,(1020, 500))
    blurred_image=blur_except_rectangle(resized_image,x_start,y_start,x_end,y_end,offset_size=[30,80],blur_kernel_size=(15,15))
    
    results=model.predict(blurred_image,conf=0.4,verbose=False,device=[0],classes=[2,3])
    # print(f"\n----------------------------------------------------------------------------------\nResult without detection \n{results[0].boxes.data}\n----------------------------------------------------------------------------------\n")
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(blurred_image, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(blurred_image, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets,object_class_id=tracker_element_handler(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, blurred_image) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                travel_direction=-1
                
                blurred_image,object_id,cx,cy,bbox_list,conf=plot_tracks(track,blurred_image)
                if f'{object_id}' in main_violation_tracker_json:
                    pass
                else:
                    main_violation_tracker_json[f'{object_id}']={}
                    # print("adding Id to tracker")
                all_id_tracker.append(object_id)
                
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         travel_direction=1
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                
                                # Fucntion for timer tracking
                                
                                time_dict=video_time_checker(count=frame_count,cap=cap)
                                
                                # Getting the name for the detected object
                                object_name=object_class_name_normalizer(object_class_id,yolo_model_classes)
                                
                                                              
                                main_violation_tracker_json[f'{object_id}']['violation_type_id']=violation_type_json['over_speed']
                                main_violation_tracker_json[f'{object_id}']['vehicle_type_id']=vehicle_class_json[f'{object_name}']
                                
                                main_violation_tracker_json[f'{object_id}']['vehicle_speed']=est_speed
                                main_violation_tracker_json[f'{object_id}']['travel_direction']=travel_direction

                                main_violation_tracker_json[f'{object_id}']['location_id']=location_id
                                main_violation_tracker_json[f'{object_id}']['camera_id']=camera_id

                                main_violation_tracker_json[f'{object_id}']['violation_date']=current_date
                                main_violation_tracker_json[f'{object_id}']['violation_time']=time_dict['time']
                                
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                # violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                # print("bbox_list-",bbox_list)
                                object_class_id
                                violation_frame_writer_original=frame_writer(frame_count,resized_image,violation_frames_record_path)
                                violation_frame_writer_blurred=frame_writer(frame_count,resized_image,violation_frames_record_path,coordiante_blur=True,bbox_list=bbox_list)
                                
                                main_violation_tracker_json[f'{object_id}']['evidence_focused_image_url']=violation_frame_writer_blurred
                                main_violation_tracker_json[f'{object_id}']['evidence_image_url']=violation_frame_writer_original
                                            
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            # print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,resized_image,all_frames_record_path)    
    frame_count+=1
    out_main.write(resized_image)
    
    # # break on pressing q or spaceq
    cv2.imshow('Strong Sort Detection detection', blurred_image)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
remove_model_from_gpu(model)
end_time=time.time()
print(f'total_time {end_time-start_time}')

2024-05-27 15:21:03.813 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


total_time 958.4203326702118


In [3]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)
maintained=json_frame_order_checker(main_violation_json)
if maintained:
    # Function for creating dir paths only
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)
    print("Debug Process 1 Completed")
    # Function for taking separating image
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)   
    print("Debug Process 2 Completed")
    
    video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path,main_violation_tracker_json)
    print("Debug Process 3 Completed")
    
    cropped_images_path_list=evidance_cropper_mt(evidance_img_dir_paths,main_violation_json)
    print("Debug Process 4 Completed")
    
    deblurred_image_paths=deblur_images(cropped_images_path_list,main_violation_json)
    print("Debug Process 6 Completed")
    
    # Using default 8 threads to achive max effiencicy
    enhance_image_path=image_enhancement_using_limit_mpx(deblurred_image_paths,num_threads=8)
    print("Debug Process 7 Completed")
    
    cropped_lp_paths=preprocssing_part_one(enhance_image_path,'models/LPD.pt',2)
    print("Debug Process 8 Completed")
    
    enhanced_cropped_images_lp=image_enhancement_using_limit_mpx(cropped_lp_paths,num_threads=8,dir_name='cropped_lp_enhanced')
    print("Debug Process 9 Completed")
    
    thresholded_image=create_threshold_imgs_mp(enhanced_cropped_images_lp)
    print("Debug Process 10 Completed")
    
    lp_csv_path=ocr_processor_mp_phase_three(folder_paths=thresholded_image,num_thread=3)
    print("Debug Process 11 Completed")
    
    ocr_json=ocr_result_filter(main_violation_tracker_json,lp_csv_path)
    print("Debug Process 12 Completed")
    
    main_violation_tracker_json=final_json_processor(main_violation_tracker_json,ocr_json)
    print("Debug Process 13 Completed")
    main_violation_tracker_json=remove_empty_from_dict(main_violation_tracker_json)
    print("Debug Process 14 Completed")
    

order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/74' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/57' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/60'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/92' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/139' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/364'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/479' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/483' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/464'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/469' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence

/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/py

CPU times: user 3min 20s, sys: 53 s, total: 4min 13s
Wall time: 48min 17s


In [4]:
main_violation_tracker_json

{'3': {'violation_type_id': 2,
  'vehicle_type_id': 4,
  'vehicle_speed': 36,
  'travel_direction': 1,
  'location_id': -1,
  'camera_id': -1,
  'violation_date': '27_05_2024',
  'violation_time': '00:00:08',
  'evidence_focused_image_url': '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/all_violation_record/223_blurred.jpg',
  'evidence_image_url': '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/all_violation_record/223.jpg',
  'evidence_video_url': '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/evidance_clips/3/evidance_3.avi',
  'numbe_plate_ocr_image_url': '/home/hlink/workspace/learning/boxmot/system_fall_back_assets/fall_back_scanned_lp.png',
  'numbe_plate_image_url': '/home/hlink/workspace/learning/boxmot/system_fall_back_assets/fall_back_normal_lp.png',
  'scanned_number_plate_number': 'MH40BP4321'},
 '4': {'violation_type_id': 2,
  'vehicle_type_id': 4,
  'vehicle_speed': 43,
  'travel_direction': 1,
  'location_id'

In [21]:
import csv
# Function to write in csv
def write_to_csv(data, file_id,filename='final_inference_output.csv',fieldnames=None):
    file_exists = os.path.exists(filename)
    if not fieldnames:
        fieldnames = ['violation_type_id','vehicle_type_id','vehicle_speed','travel_direction','location_id','camera_id','evidence_image_url','number_plate_ocr_image_url','evidence_video_url','number_plate_image_url','scanned_number_plate_number','violation_date','violation_time']
    with open(filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()  # Write header only if the file is newly created
       
        # # Find the entry corresponding to the file_id and write it to the CSV
        # for entry in data:
        #     print(entry)
        #     if file_id in entry:  # Check if the file_id exists in the current entry
        #         print(file_id)
        #         writer.writerow(entry[file_id])
# Function to iterate through JSON
def csv_writer(csv_file_name,all_violation_records_json):
    if os.path.basename(csv_file_name).endswith('.csv'):
         pass
    else:
        csv_file_name=str(csv_file_name)+'.csv'
        
    for idx in all_violation_records_json:
            write_to_csv(all_violation_records_json,str(idx),csv_file_name) 
    return os.path.abspath(csv_file_name)

In [22]:
final_csv_file_name=current_date+'inference_results.csv'
csv_writer(f'{final_csv_file_name}',main_violation_tracker_json)

'/home/hlink/workspace/learning/boxmot/27_05_2024inference_results.csv'

In [28]:
# Write to CSV
final_csv_path=json_to_csv(main_violation_tracker_json, order, f'{final_csv_file_name}',debug=True)